## Expérimentation pour préparer la métrique LLM as a judge

<div class="alert alert-warning" role="alert">
<h3 class="alert-heading">Warning</h3>
Ce _notebook_ est un brouillon pour l'évaluation de la qualité du retrieval. Pas eu le temps de finir car c'est les vacances 🏖️
</div>

D'abord un paquet de code extrait de `run_pipeline` pour avoir un environnement avec les éléments suivants:

- La base de données vectorielle (à venir: on récupèrera direct la BDD complète de S3)
- Le retriever brut
- Le retriever avec reranker 

Le bloc de code est long mais y a des choses exploratoires en dessous

In [1]:
import argparse
import ast
import logging
import os
import shutil
import subprocess
import tempfile
from pathlib import Path

import mlflow
import pandas as pd
import s3fs
from langchain_core.prompts import PromptTemplate

from src.chain_building import build_chain_validator
from src.chain_building.build_chain import build_chain
from src.config import CHATBOT_TEMPLATE, CHROMA_DB_LOCAL_DIRECTORY, RAG_PROMPT_TEMPLATE, S3_BUCKET
from src.db_building import build_vector_database, chroma_topk_to_df, load_retriever
from src.evaluation import answer_faq_by_bot, compare_performance_reranking, evaluate_question_validator, transform_answers_bot
from src.model_building import build_llm_model

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": f"""https://{os.environ["AWS_S3_ENDPOINT"]}"""})

# INPUT: FAQ THAT WILL BE USED FOR EVALUATION -----------------
bucket = "projet-llm-insee-open-data"
path = "data/FAQ_site/faq.parquet"
faq = pd.read_parquet(f"{bucket}/{path}", filesystem=fs)
# Extract all URLs from the 'sources' column
faq["urls"] = faq["sources"].str.findall(r"https?://www\.insee\.fr[^\s]*").apply(lambda s: ", ".join(s))


data_raw_s3_path= "data/raw_data/applishare_solr_joined.parquet"
collection_name = "insee_data"
embedding_model = "OrdalieTech/Solon-embeddings-large-0.1"
db, df_raw = build_vector_database(
            data_path=data_raw_s3_path,
            persist_directory=CHROMA_DB_LOCAL_DIRECTORY,
            collection_name=collection_name,
            filesystem=fs,
            chunk_size = 512,
            chunk_overlap = 100,
            max_pages = 20,
            embedding_model = embedding_model
        )

llm, tokenizer = build_llm_model(
            model_name=os.getenv("LLM_MODEL_NAME", "mistralai/Mistral-7B-Instruct-v0.2"),
            quantization_config=True,
            config=True,
            token=os.getenv("HF_TOKEN"),
            streaming=False,
            generation_args=None,
        )

embedding_model = "OrdalieTech/Solon-embeddings-large-0.1"
retriever, vectorstore = load_retriever(
            emb_model_name="OrdalieTech/Solon-embeddings-large-0.1",
            vectorstore=db,
            persist_directory=CHROMA_DB_LOCAL_DIRECTORY,
            retriever_params={"search_type": "similarity", "search_kwargs": {"k": 30}},
        )

validator = build_chain_validator(evaluator_llm=llm, tokenizer=tokenizer)
validator_answers = evaluate_question_validator(validator=validator)
true_positive_validator = validator_answers.loc[validator_answers["real"], "real"].mean()
true_negative_validator = 1 - (validator_answers.loc[~validator_answers["real"], "real"].mean())


# Define a langchain prompt template
RAG_PROMPT_TEMPLATE_RERANKER = tokenizer.apply_chat_template(
    CHATBOT_TEMPLATE, tokenize=False, add_generation_prompt=True
)
prompt = PromptTemplate(
    input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE_RERANKER
)

reranking_method = "BM25"
chain = build_chain(
                    retriever=retriever,
                    prompt=prompt,
                    llm=llm,
                    bool_log=False,
                    reranker=reranking_method,
                )

2024-07-26 13:18:31,728 - INFO - Loading faiss with AVX512 support.
2024-07-26 13:18:31,752 - INFO - Successfully loaded faiss with AVX512 support.


In [50]:
import pandas as pd

with fs.open("projet-llm-insee-open-data/data/eval_data/eval_dataset.csv", 'rb') as f:
    eval_dataset = pd.read_csv(f)

eval_dataset.iloc[0]

context               Près de 200 000 habitants résident sur le Terr...
question              Quelle était la population du Territoire de la...
answer                La population du Territoire de la Côte Ouest é...
source_doc                 https://www.insee.fr/fr/statistiques/1293858
groundedness_score                                                    4
groundedness_eval     Le contexte fournit des informations sur la po...
relevance_score                                                     5.0
relevance_eval        Cette question est très utile pour les agents ...
standalone_score                                                    5.0
standalone_eval       La question demande des informations spécifiqu...
Name: 0, dtype: object

In [122]:
web4g = pd.read_parquet(f"projet-llm-insee-open-data/{data_raw_s3_path}", filesystem=fs)
valid_urls = web4g['url'].unique().tolist()

## Evaluation du potentiel hallucinatoire sans RAG

Idée: c'est le baseline : si on fait pas de RAG, est-ce qu'on est mauvais ? Pour ça, on peut poser des questions à un LLM non entraîné et vérifier les sources qu'il nous donne: existence (check si URL existe) voire qualité (?) (même catégorie du site que ce qu'on attend ?)

In [131]:
prompt_no_context = f"""
<s>[INST]
Tu es un assistant spécialisé dans la statistique publique répondant aux questions d'agents de l'INSEE.
Réponds en Français exclusivement. 
Donne une URL sur le site insee.fr. Si tu proposes des étapes de navigation sur le site, donne l'URL final. Vérifie que les URL que tu proposes existent réellement.

---
Voici la question à laquelle tu dois répondre :
Question: {question}
[/INST]
"""

answer_no_context = llm.invoke(prompt_no_context)

In [114]:
relevant_docs = eval_dataset['context'].tolist()[:1]
question = eval_dataset['question'].tolist()[:1]

# Forcer le contexte
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

prompt_context_only_good_document = RAG_PROMPT_TEMPLATE.format(question=question, context=context)
print(prompt_context_only_good_document)


<s>[INST]
Tu es un assistant spécialisé dans la statistique publique répondant aux questions d'agent de l'INSEE.
Réponds en Français seulement.
Utilise les informations obtenues dans le contexte, réponds de manière argumentée à la question posée.
La réponse doit être développée et citer ses sources.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.
Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: 
Extracted documents:
Document 0:::
Près de 200 000 habitants résident sur le Territoire de la Côte Ouest au 1er janvier 2006. La population a continué sa croissance au rythme de 1,5 % par an depuis 1999 sous le seul effet d'un excédent de naissances sur les décès. Pour la première fois depuis longtemps les migrations se soldent par un résultat nul. L'habitat s'est légèrement modifié : plus de collectif et moins d'habitat traditionnel, une taille de logement recentrée autour des trois et quatre pièces, une offre locative privée qui s'est étoffée. Toutefois le

In [132]:
import re
import requests
# Extract all URLs using regular expressions
urls = re.findall(r'<(https://www\.insee\.fr[^>]+)>', answer_no_context)

# Function to check if URLs exist
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return f"{url} exists and is reachable."
        else:
            return f"{url} returned status code {response.status_code}."
    except requests.exceptions.RequestException as e:
        return f"{url} could not be reached. Error: {e}"

# Check each URL and print the result
results = [check_url(url) for url in urls]

results

['https://www.insee.fr/fr/statistiques exists and is reachable.',
 'https://www.insee.fr/fr/statistiques/fichier/1110003/tableau/T1_POP_1110003.xls returned status code 500.']

Pour le moment, je n'utilise pas `prompt_context_only_good_document` car je pense qu'il s'agit d'une métrique pour évaluer la qualité de la génération, pas du retrieval. Je pense que ça vaut le coup de distinguer ces deux niveaux de contrôle qualité.

In [90]:
def build_chain2(hf_embeddings, vectorstore, retriever, prompt, llm):
    """ 
    Build a LLM chain based on Langchain package and INSEE data 
    """
    #Create a Langchain LLM Chain 
    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm 
        | StrOutputParser()
    )
    return rag_chain_from_docs

## Evaluation sans/avec reranking

Le bout de code ci-dessous vise à fournir un premier canevas pour évaluer la capacité du modèle à faire du bon _retrieval_ sans ou avec reranking.  

C'est pas fini mais y a les vacances qui arrivent. 

In [85]:
llm = llm
knowledge_index = vectorstore
output_file = "test_generated_ans.json"
reranker = None
verbose=True
test_settings = os.getenv("LLM_MODEL_NAME", "mistralai/Mistral-7B-Instruct-v0.2")
batch_size = 5

batch_questions = eval_dataset['question'].iloc[:5].tolist()

answer_no_reranking_context_complete = []
answer_reranking_context_complete = []
for questions in batch_questions:
    answer_no_reranking_context_complete.append(
        retriever.invoke(questions)
    )
    answer_reranking_context_complete.append(
        chain.invoke(questions)
    )

In [88]:
idx = 0
def check_answer_is_expected(answer_no_reranking_context_complete, idx = 0, k = 5):
    if isinstance(answer_no_reranking_context_complete, dict):
        answer_no_reranking_context_complete = answer_no_reranking_context_complete['context']
    retrieved_docs = answer_no_reranking_context_complete[idx]
    for docs in retrieved_docs:
        result_list = []
        for doc in retrieved_docs:
            row = {"page_content": doc.page_content}
            row.update(doc.metadata)
            result_list.append(row)
    retrieved_docs_df = pd.DataFrame(result_list)
    retrieved_docs_df['expected_url'] = (retrieved_docs_df['url'] == eval_dataset['source_doc'][idx])
    retrieved_docs_df['question'] = questions[idx]
    return retrieved_docs_df.head(k)

answer_no_reranking_context_complete = check_answer_is_expected(answer_no_reranking_context_complete)
answer_reranking_context_complete = check_answer_is_expected(answer_reranking_context_complete)

AttributeError: 'str' object has no attribute 'page_content'